In [1]:
import os
import re
from typing import List, Dict, Optional
from openai import OpenAI
from dotenv import load_dotenv

# === Setup ===
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"))

# === Utility Functions ===

def llm_call(prompt: str, model: str = "gpt-4") -> str:
    """Basic LLM call wrapper"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

def extract_xml(text: str, tag: str) -> str:
    """Extract content between XML-style tags"""
    pattern = rf"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

def parse_tasks(xml: str) -> List[Dict]:
    """Parses <task> XML blocks into dictionaries"""
    tasks = []
    current_task = {}

    for line in xml.splitlines():
        line = line.strip()
        if line.startswith("<task>"):
            current_task = {}
        elif line.startswith("<type>"):
            current_task["type"] = line[6:-7].strip()
        elif line.startswith("<description>"):
            current_task["description"] = line[12:-13].strip()
        elif line.startswith("</task>"):
            if "description" in current_task:
                if "type" not in current_task:
                    current_task["type"] = "default"
                tasks.append(current_task)

    return tasks

# === Worker Agent Base Class ===

class WorkerAgent:
    def __init__(self, task_type: str):
        self.task_type = task_type

    def run(self, original_task: str, task_description: str) -> str:
        raise NotImplementedError("Must implement run() method in subclass")

# === Worker Agent Implementations ===

class ZoningAgent(WorkerAgent):
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a zoning permit expert.

Main Task: {original_task}
Subtask Type: {self.task_type}
Subtask Description: {task_description}

<response>
- Explain the purpose of this step.
- Describe required documents or portals.
- Note any blockers like zoning conflicts or variances.
</response>
"""
        raw_output = llm_call(prompt)
        result = extract_xml(raw_output, "response")
        if not result:
            print(f"[WARNING] No <response> tag found. Using raw output:\n{raw_output}")
            result = raw_output
        return result

class SitePlanAgent(WorkerAgent):
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a site planning advisor.

Main Task: {original_task}
Subtask Type: {self.task_type}
Subtask Description: {task_description}

<response>
- What does site planning involve?
- What approvals or drawings are needed?
- Who reviews them and why might they reject it?
</response>
"""
        raw_output = llm_call(prompt)
        result = extract_xml(raw_output, "response")
        if not result:
            print(f"[WARNING] No <response> tag found. Using raw output:\n{raw_output}")
            result = raw_output
        return result

class GenericAgent(WorkerAgent):
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a permitting assistant.

Main Task: {original_task}
Subtask Type: {self.task_type}
Subtask Description: {task_description}

<response>
- Describe this subtask.
- List how to execute it.
- Include potential challenges.
</response>
"""
        raw_output = llm_call(prompt)
        result = extract_xml(raw_output, "response")
        if not result:
            print(f"[WARNING] No <response> tag found. Using raw output:\n{raw_output}")
            result = raw_output
        return result

# === Orchestrator ===

class Orchestrator:
    def __init__(self, orchestrator_prompt: str):
        self.orchestrator_prompt = orchestrator_prompt

    def get_worker(self, task_type: str) -> WorkerAgent:
        type_lower = task_type.lower()
        if "zoning" in type_lower:
            return ZoningAgent(task_type)
        elif "site" in type_lower:
            return SitePlanAgent(task_type)
        else:
            return GenericAgent(task_type)

    def process(self, task: str) -> Dict:
        # Step 1: Decompose task using orchestrator LLM
        orchestrator_input = self.orchestrator_prompt.format(task=task)
        response = llm_call(orchestrator_input)
        print("\n[Raw Orchestrator Output]\n", response)

        analysis = extract_xml(response, "analysis")
        tasks_xml = extract_xml(response, "tasks")
        tasks = parse_tasks(tasks_xml)

        print("\n=== ORCHESTRATOR ===")
        print("Analysis:", analysis)
        print("Parsed Tasks:", tasks)

        # Step 2: Dispatch tasks to worker agents
        results = []
        for task_info in tasks:
            agent = self.get_worker(task_info["type"])
            result = agent.run(task, task_info["description"])
            print(f"\n=== {task_info['type'].upper()} RESULT ===\n{result}")
            results.append({
                "type": task_info["type"],
                "description": task_info["description"],
                "result": result
            })

        return {
            "analysis": analysis,
            "worker_results": results
        }

# === Prompt Template for Orchestrator ===

orchestrator_prompt = """
You are a permitting expert. Your job is to analyze a high-level construction request and break it into permitting-related subtasks.

<analysis>
Provide an overall explanation of what steps are involved.
</analysis>

<tasks>
Provide 2–4 <task> entries with a <type> and <description>. Use this format:
<task>
  <type>zoning</type>
  <description>Verify land is zoned for commercial use.</description>
</task>
</tasks>

Task: {task}
"""

# === Main Runner ===

if __name__ == "__main__":
    user_prompt = "What permits do I need to add a pool to my house in The Woodlands Texas?"

    orchestrator = Orchestrator(orchestrator_prompt)
    results = orchestrator.process(user_prompt)

    print("\n\n=== FINAL REPORT ===")
    print("Analysis:\n", results["analysis"])
    for r in results["worker_results"]:
        print(f"\n--- {r['type'].upper()} ---")
        print("Description:", r["description"])
        print("Result:\n", r["result"])



[Raw Orchestrator Output]
 <analysis>
Adding a pool to your house in The Woodlands, Texas, involves several steps related to permitting. First, you need to verify the zoning of your property to ensure that a pool is allowed. Second, you need to apply for a building permit from the local authority, which may require submitting detailed plans of the proposed pool and your property. Third, you may need to obtain a plumbing permit if the pool will be connected to your home's plumbing system. Finally, you may need to get an electrical permit if the pool will have lights or other electrical features.
</analysis>

<tasks>
<task>
  <type>Zoning</type>
  <description>Verify that your property is zoned for residential use and that a pool is allowed under the zoning regulations.</description>
</task>
<task>
  <type>Building Permit</type>
  <description>Apply for a building permit from the local authority. This will likely require submitting detailed plans of the proposed pool and your property.<

In [6]:
import os
import re
from typing import List, Dict, Optional
from openai import OpenAI
from dotenv import load_dotenv

# === Setup ===
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"))

# === Utility Functions ===

def llm_call(prompt: str, model: str = "gpt-4") -> str:
    """Basic LLM call wrapper."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

def extract_xml(text: str, tag: str) -> str:
    """Extract content between XML-style tags."""
    pattern = rf"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

def parse_tasks(xml: str) -> List[Dict]:
    """Parses <task> XML blocks into dictionaries."""
    tasks = []
    current_task = {}

    for line in xml.splitlines():
        line = line.strip()
        if line.startswith("<task>"):
            current_task = {}
        elif line.startswith("<type>"):
            current_task["type"] = line[6:-7].strip()
        elif line.startswith("<description>"):
            # FIX: Avoid leaving leading/trailing angle brackets in the string
            value = line[13:-14].strip()
            if value.startswith(">") and value.endswith("<"):
                value = value[1:-1].strip()
            current_task["description"] = value
        elif line.startswith("</task>"):
            if "description" in current_task:
                if "type" not in current_task:
                    current_task["type"] = "default"
                tasks.append(current_task)
    return tasks

# === Worker Agent Base Class ===

class WorkerAgent:
    """Abstract base class for all specialized worker agents."""
    def __init__(self, task_type: str):
        self.task_type = task_type

    def run(self, original_task: str, task_description: str) -> str:
        raise NotImplementedError("The 'run' method must be implemented in a subclass.")

# === Worker Agent Implementations ===

enforce_message = """IMPORTANT:

- Your entire answer MUST be wrapped in a single <response>...</response> block.
- Do NOT include any explanations or notes outside the <response> block.
- Do NOT add markdown, extra lines, or any text before or after <response>.
- The output will be parsed automatically — format it EXACTLY as shown.

Example format:

"""

class HematologyAgent(WorkerAgent):
    """Worker that analyzes blood cell counts (Complete Blood Count)."""
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a hematology analysis expert. Your task is to interpret the blood count section of a lab report.

Main Task: {original_task}
Your Subtask: {task_description}

{enforce_message}

<response>
- Explain the purpose of analyzing these blood values.
- Identify any out-of-range values (e.g., high/low RBC, WBC, Platelets).
- Briefly note the potential clinical significance of any abnormalities.
</response>
"""
        raw_output = llm_call(prompt)
        print(f"\n[Raw LLM Output for {self.task_type}]\n{raw_output}")
        if "<response>" not in raw_output:
            raise ValueError(f"Expected <response> in LLM output:\n{raw_output}")
        return extract_xml(raw_output, "response")

class RenalFunctionAgent(WorkerAgent):
    """Worker that analyzes kidney function markers."""
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a renal function analysis expert. Your task is to interpret the kidney-related markers from a lab report.

Main Task: {original_task}
Your Subtask: {task_description}

{enforce_message}

<response>
- Explain the purpose of analyzing these kidney markers.
- Identify any out-of-range values (e.g., Creatinine, BUN, GFR).
- Briefly note the potential clinical significance of any abnormalities.
</response>
"""
        raw_output = llm_call(prompt)
        print(f"\n[Raw LLM Output for {self.task_type}]\n{raw_output}")
        if "<response>" not in raw_output:
            raise ValueError(f"Expected <response> in LLM output:\n{raw_output}")
        return extract_xml(raw_output, "response")

class LiverFunctionAgent(WorkerAgent):
    """Worker that analyzes liver enzyme markers."""
    def run(self, original_task: str, task_description: str) -> str:
        prompt = f"""
You are a liver function analysis expert. Your task is to interpret the liver enzyme section of a lab report.

Main Task: {original_task}
Your Subtask: {task_description}

{enforce_message}

<response>
- Explain the purpose of analyzing these liver enzymes.
- Identify any out-of-range values (e.g., ALT, AST, ALP).
- Briefly note the potential clinical significance of any abnormalities.
</response>
"""
        raw_output = llm_call(prompt)
        print(f"\n[Raw LLM Output for {self.task_type}]\n{raw_output}")
        if "<response>" not in raw_output:
            raise ValueError(f"Expected <response> in LLM output:\n{raw_output}")
        return extract_xml(raw_output, "response")

# === Orchestrator ===

class Orchestrator:
    def __init__(self, orchestrator_prompt: str):
        self.orchestrator_prompt = orchestrator_prompt

    ############################################################################
    ##                                                                        ##
    ##              [  CHALLENGE: IMPLEMENT THE WORKER DISPATCHER ]            ##
    ##                                                                        ##
    ##  Your task is to implement this method. It must look at the incoming   ##
    ##  `task_type` (a string like "hematology" or "renal") and return an     ##
    ##  instance of the correct specialized worker agent.                     ##
    ##                                                                        ##
    ############################################################################
    def get_worker(self, task_type: str) -> WorkerAgent:
        """Inspects the task type and returns the correct specialized agent."""
        type_norm = re.sub(r"[^a-z0-9 ]+", "", task_type.lower())
    
        hematology_terms = ["cbc", "blood count", "hematology"]
        renal_terms = ["renal", "kidney"]
        liver_terms = ["liver", "hepatic"]
    
        if any(term in type_norm for term in hematology_terms):
            return HematologyAgent(task_type="hematology")
        if any(term in type_norm for term in renal_terms):
            return RenalFunctionAgent(task_type="renal")
        if any(term in type_norm for term in liver_terms):
            return LiverFunctionAgent(task_type="liver")
    
        # Optional fallback to LLM
        inferred = classify_task_type(task_type)
        return self.get_worker(inferred)  # recursive retry
    
    def classify_task_type(description: str) -> str:
        """Mini LLM fallback classifier to handle ambiguous or unrecognized types."""
        prompt = f"""
Classify this lab panel type into one of: hematology, renal, liver.

Panel type: {description}

Respond with only one word: hematology, renal, liver, or unknown.
"""
        return llm_call(prompt, model="gpt-3.5-turbo").lower()

    def process(self, task: str) -> Dict:
        """Runs the full Orchestrator-Workers workflow."""
        orchestrator_input = self.orchestrator_prompt.format(task=task)
        response = llm_call(orchestrator_input)
        print("\n[Raw Orchestrator Output]\n", response)

        analysis = extract_xml(response, "analysis")
        tasks_xml = extract_xml(response, "tasks")
        tasks = parse_tasks(tasks_xml)

        print("\n=== ORCHESTRATOR ANALYSIS & PLAN ===")
        print("Analysis:", analysis)
        print("Parsed Tasks:", tasks)

        results = []
        for task_info in tasks:
            try:
                agent = self.get_worker(task_info["type"])
                result = agent.run(task, task_info["description"])
                print(f"\n=== {task_info['type'].upper()} RESULT ===\n{result}")
                results.append({
                    "type": task_info["type"],
                    "description": task_info["description"],
                    "result": result
                })
            except ValueError as e:
                print(f"\n--- ERROR --- \n{e}")

        return {"analysis": analysis, "worker_results": results}

# === Prompt Template for Orchestrator ===

orchestrator_prompt = """
You are a clinical lab data analyst. Your job is to analyze a set of patient lab results and create a plan to interpret them systematically.

The plan must be broken down into subtasks, one for each major panel in the lab report.

Return your response in the following format, with an <analysis> section and a <tasks> section.

<analysis>
Provide a high-level summary of the lab panels present and the overall goal of the interpretation.
</analysis>

<tasks>
Provide one <task> entry for each major lab panel found in the data. Each task must have a <type> and a <description>.
Example task format:
<task>
  <type>hematology</type>
  <description>Analyze the Complete Blood Count (CBC) panel, including RBC, WBC, and platelets.</description>
</task>
</tasks>

Here is the high-level task and data:
Task: {task}
"""

# === Main Runner ===

if __name__ == "__main__":
    lab_results_data = """
    Patient Lab Report:
    - Panel: Complete Blood Count (CBC)
      - White Blood Cell (WBC): 11.5 x10^9/L (Normal: 4.5-11.0)
      - Red Blood Cell (RBC): 4.6 x10^12/L (Normal: 4.2-5.4)
      - Platelets: 140 x10^9/L (Normal: 150-450)
    - Panel: Renal Function Panel
      - Creatinine: 1.4 mg/dL (Normal: 0.6-1.2)
      - BUN: 25 mg/dL (Normal: 7-20)
    - Panel: Liver Function Panel
      - ALT: 55 U/L (Normal: 7-56)
      - AST: 60 U/L (Normal: 10-40)
    """
    
    user_prompt = f"Please interpret the following lab results and provide a summary: {lab_results_data}"

    orchestrator = Orchestrator(orchestrator_prompt)
    final_report = orchestrator.process(user_prompt)

    print("\n\n=== FINAL INTERPRETATION REPORT ===")
    print("Overall Analysis:\n", final_report.get("analysis", "N/A"))
    for r in final_report.get("worker_results", []):
        print(f"\n--- {r['type'].upper()} PANEL ---")
        print("Task Description:", r["description"])
        print("Interpretation:\n", r["result"])



[Raw Orchestrator Output]
 <analysis>
The lab report provided includes results from three major panels: Complete Blood Count (CBC), Renal Function Panel, and Liver Function Panel. The overall goal of the interpretation is to assess the patient's health status by comparing the lab results with the normal ranges. Any significant deviations from the normal ranges may indicate potential health issues related to the blood, kidneys, or liver.
</analysis>

<tasks>
<task>
  <type>Complete Blood Count (CBC)</type>
  <description>Analyze the Complete Blood Count (CBC) panel, including White Blood Cell (WBC), Red Blood Cell (RBC), and Platelets. Compare the patient's results with the normal ranges to identify any abnormalities that could indicate infection, anemia, or clotting disorders.</description>
</task>

<task>
  <type>Renal Function Panel</type>
  <description>Analyze the Renal Function Panel, including Creatinine and Blood Urea Nitrogen (BUN). Compare the patient's results with the norma